<a href="https://colab.research.google.com/github/Drashti30/Low_Level_Design/blob/main/System_Design_Practice1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from abc import ABC, abstractmethod
from datetime import time
from collections import defaultdict

# ---------- Domain model ----------

class Animal(ABC):
    def __init__(self, name: str, weight_kg: float):
        self.name = name
        self.weight_kg = weight_kg

    @property
    @abstractmethod
    def species(self) -> str: ...

    @abstractmethod
    def sound(self) -> str: ...

    @abstractmethod
    def diet(self) -> str: ...   # "carnivore" | "herbivore" | "omnivore"

    def feed(self, food: str) -> str:
        # simple dietary rule; could be replaced with strategy later
        allowed = {
            "carnivore": {"meat", "fish"},
            "herbivore": {"grass", "leaves", "fruits"},
            "omnivore":  {"meat", "fish", "grass", "leaves", "fruits"}
        }
        if food not in allowed[self.diet()]:
            return f"{self.name} ({self.species}) refuses {food}."
        return f"{self.name} ({self.species}) eats {food}."

    def make_sound(self) -> str:
        return f"{self.name} ({self.species}) says {self.sound()}!"

class Lion(Animal):
    @property
    def species(self) -> str: return "Lion"
    def sound(self) -> str: return "ROAR"
    def diet(self) -> str: return "carnivore"

class Elephant(Animal):
    @property
    def species(self) -> str: return "Elephant"
    def sound(self) -> str: return "Pawoo"
    def diet(self) -> str: return "herbivore"

class Monkey(Animal):
    @property
    def species(self) -> str: return "Monkey"
    def sound(self) -> str: return "Oo-oo-aa-aa"
    def diet(self) -> str: return "omnivore"


class Enclosure:
    """A physical area that holds animals (composition)."""
    def __init__(self, name: str, capacity: int):
        self.name = name
        self.capacity = capacity
        self._animals: list[Animal] = []

    def add(self, animal: Animal) -> bool:
        if len(self._animals) >= self.capacity:
            return False
        self._animals.append(animal)
        return True

    def remove(self, name: str) -> bool:
        for i, a in enumerate(self._animals):
            if a.name == name:
                self._animals.pop(i)
                return True
        return False

    def animals(self) -> list[Animal]:
        return list(self._animals)  # copy for safety


class Zoo:
    """High-level facade that manages enclosures, feedings and queries."""
    def __init__(self, name: str):
        self.name = name
        self._enclosures: dict[str, Enclosure] = {}
        self._feeding_schedule: dict[time, list[tuple[str, str, str]]] = defaultdict(list)
        # time -> list of (enclosure_name, animal_name, food)

    # --- enclosure management ---
    def add_enclosure(self, enc: Enclosure) -> None:
        self._enclosures[enc.name] = enc

    def move(self, animal_name: str, from_enc: str, to_enc: str) -> bool:
        if from_enc not in self._enclosures or to_enc not in self._enclosures:
            return False
        # remove from 'from_enc'
        for a in self._enclosures[from_enc].animals():
            if a.name == animal_name:
                if self._enclosures[to_enc].add(a):
                    self._enclosures[from_enc].remove(animal_name)
                    return True
        return False

    # --- queries ---
    def all_animals(self) -> list[Animal]:
        res = []
        for enc in self._enclosures.values():
            res.extend(enc.animals())
        return res

    def find_by_species(self, species: str) -> list[Animal]:
        return [a for a in self.all_animals() if a.species == species]

    def chorus(self) -> list[str]:
        return [a.make_sound() for a in self.all_animals()]

    # --- feeding ---
    def schedule_feeding(self, at: time, enclosure_name: str, animal_name: str, food: str) -> None:
        self._feeding_schedule[at].append((enclosure_name, animal_name, food))

    def run_feedings(self, at: time) -> list[str]:
        logs = []
        for enc_name, animal_name, food in self._feeding_schedule.get(at, []):
            enc = self._enclosures.get(enc_name)
            if not enc:
                logs.append(f"[{at}] Enclosure '{enc_name}' not found.")
                continue
            match = next((a for a in enc.animals() if a.name == animal_name), None)
            if not match:
                logs.append(f"[{at}] Animal '{animal_name}' not found in {enc_name}.")
                continue
            logs.append(f"[{at}] {match.feed(food)}")
        return logs

# ---------- Example usage (would be in a separate main/test) ----------

savannah = Enclosure("Savannah", capacity=3)
jungle   = Enclosure("Jungle", capacity=4)

simba = Lion("Simba", 180)
nala  = Lion("Nala", 160)
dumbo = Elephant("Dumbo", 540)
momo  = Monkey("Momo", 35)

savannah.add(simba); savannah.add(nala)
jungle.add(dumbo); jungle.add(momo)

z = Zoo("City Zoo")
z.add_enclosure(savannah)
z.add_enclosure(jungle)

# Queries
_ = z.chorus()                  # each animal makes its sound
lions = z.find_by_species("Lion")

# Feeding schedule + execution
z.schedule_feeding(time(9, 0), "Savannah", "Simba", "meat")
z.schedule_feeding(time(9, 0), "Jungle",   "Dumbo", "grass")
z.schedule_feeding(time(9, 0), "Jungle",   "Momo",  "fruits")

logs = z.run_feedings(time(9, 0))
for line in logs:
    print(line)


[09:00:00] Simba (Lion) eats meat.
[09:00:00] Dumbo (Elephant) eats grass.
[09:00:00] Momo (Monkey) eats fruits.
